## machine learning

imports:

In [119]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.layers import SimpleRNN, Reshape
from keras.optimizers import Adam
import numpy as np
import pandas as pd
import h5py
from numba import jit
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
import simcat
import toytree
from copy import deepcopy
import ipyparallel as ipp
from toyplot import matrix

## Neural network on Iris data:

Iris dataset has four predictor variables for three species.

### First load in the data:

In [49]:
training_iris = pd.read_csv("iris_training.csv")

In [50]:
test_iris = pd.read_csv("iris_test.csv")

Look at the data:

In [52]:
training_iris[:5]

,SepalLength,SepalWidth,PentalLength,PentalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


### Prepare our training data:

In [68]:
# this gives us integer labels
trainY = np.array(training_iris['Species'],dtype=np.int8)
# one hot encode the integer labels
encoding_trainY = np_utils.to_categorical(trainY)

In [84]:
# this gives us the raw, continuous predictor data
trainX = np.array(training_iris)[:,:-1]

# now let's scale this for the model
scaler=StandardScaler()
scaler.fit(trainX)
scaled_trainX=scaler.transform(trainX)

### Prepare our test data the same way:

In [102]:
testY = np.array(test_iris['Species'],dtype=np.int8)
encoding_testY = np_utils.to_categorical(testY)

testX = np.array(test_iris)[:,:-1]
scaler=StandardScaler()
scaler.fit(testX)
scaled_testX=scaler.transform(testX)

### Now define a small neural network using Keras:

In [97]:
# Create a model -- lots of options in what we do here, except that the input dimensions
# and output dimensions are both fixed -- inputs at 1x4, outputs at 1x3.
# This model has a single hidden layer of 3 nodes.
model = Sequential()
model.add(Dense(3, input_dim=4, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [98]:
# we have to compile our model before use -- using an Adam optimizer and 
# calculating loss using categorical crossentropy: 
model.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=['accuracy'])

### Fit the network:

In [101]:
# now we can fit our model! Let's just start with 10 epochs to keep things quick
model.fit(scaled_trainX, encoding_trainY, epochs=300, batch_size=5,verbose=False)

### See how well the model can classify the test data:

In [103]:
# get accuracy on the test set
scores = model.evaluate(scaled_testX, encoding_testY)
print("\nAccuracy: %.2f%%" % (scores[1]*100))
# plot the confusion matrix
matrix(confusion_matrix(np.argmax(encoding_testY,axis=1),np.argmax(model.predict(scaled_testX),axis=1)))

30/30 [==============================] - 0s 5ms/step

Accuracy: 96.67%


(<toyplot.canvas.Canvas at 0x1a852d3310>,
 <toyplot.coordinates.Table at 0x1a852d3290>)

0 1 2 0 8.0 0.0 0.0 1 0.0 14.0 0.0 2 0.0 1.0 7.0

### looks like our model classified just one of the test samples incorrectly.

# Moving on to simcat:

We want to be able to infer where introgression happened on a tree given a bunch of structured sequence data and a topology. To do this, we will simulate introgression on a **given** topology under a bunch of possible introgression scenarios. For simplicity, we will not change branch lengths.

In [5]:
## generate a random tree
tree = toytree.rtree.unittree(ntips=6, treeheight=3, seed=12345)
c, a = tree.draw(tree_style='c',node_labels=tree.get_node_values('name',show_root=True,show_tips=True))

r5 r4 r3 r2 r1 r0 i6 i7 i8 i9 i10 0 1 2 3

# Simulating data

# **Do not run this** -- only needs to have been done once  
(and I've done it for you)

Creating a database of demographic scenario labels:

In [6]:
## don't run this! Proof of concept only
## init a database
db1 = simcat.DataBase(
    name="6taxa", 
    workdir="./databases", 
    tree=tree, 
    nedges=1,
    ntrees=200,
    ntests=1,
    nreps=2,
    edge_function=None,
    constrained_times=1,
    mig_rate_bounds = [.05,.3],
    force=True)

stored 18400 labels to /Volumes/My Passport/sims/databases/6taxa.hdf5


In [7]:
## don't run this! Proof of concept only
ipyclient = ipp.Client()

In [8]:
## don't run this! Proof of concept only
ipyclient

Now fill a database with simulated sequence data:

In [9]:
## don't run this! Proof of concept only
## filling the database
db1.run(ipyclient)

host compute node: [4 cores] on Patricks-MBP.fios-router.home
[                    ]   0% | 0:00:00 | simulating count matrices

submitting jobs


[                    ]   0% | 0:07:51 | simulating count matrices

Done with round: 0 of 5


[                    ]   0% | 0:15:59 | simulating count matrices

...

Done with round: 2 of 5


[                    ]   0% | 0:31:34 | simulating count matrices

Done with round: 3 of 5


[                    ]   0% | 0:37:20 | simulating count matrices

Done with round: 4 of 5


# Now pick back up here:  
### reading in the data

In [104]:
dat=h5py.File("databases/6taxa.hdf5")

### Prepare the labels

These are the "answers" accompanying the simulated raw data.

In [105]:
# save sources and target labels into separate objects
sources=dat['admix_sources']
targets=dat['admix_targets']
# combine into a single array
combo=np.hstack([sources,targets])

# combine across columns
class_ids=np.array([str(x) for x in combo])

# make a dict assigning each source / target pair an integer
id_dict=dict(enumerate(np.unique(class_ids)))
inv_dict = {v: k for k, v in id_dict.iteritems()}

# make a set of integer labels to go alongside the data
class_ids_int = np.zeros(class_ids.shape,dtype=np.int32)
counter = 0
for str_id in class_ids:
    class_ids_int[counter] = inv_dict[str_id]
    counter += 1
# here's our (near) final set of y values
y = class_ids_int

### prepare the raw data

In [106]:
# make a function to flatten the count objects from 15x16x16
def flattendb(counts):
    newshape=reduce(lambda x,y: x*y, counts[0].shape)
    X = np.zeros((counts.shape[0],newshape))
    for i in range(len(counts)):
        X[i] = counts[i].ravel()
    return(X)

In [107]:
# here's our final set of x values
X = flattendb(dat['counts'])

### Now separate data sets into training and testing

In [108]:
# separate our data into training and test sets
trainX, testX, trainY, testY = train_test_split(X, y,
	test_size = .2, random_state = 42)

In [109]:
# whoops, still need to one-hot encode our integer labels for model fitting
encoding_trainY = np_utils.to_categorical(trainY)
encoding_testY = np_utils.to_categorical(testY)

### Define our neural network using Keras:

How do we decide what model structure to use? The answer I'm finding is typically some version of "Ultimately, the selection of an architecture for your neural network will come down to trial and error." (https://www.heatonresearch.com/2017/06/01/hidden-layers.html)

In [24]:
# Create a model -- lots of options in what we do here, except that the input dimensions
# and output dimensions are both fixed -- inputs at 1x3840, outputs at 1x46.
# This model has a single hidden layer of 1024 nodes.
model = Sequential()
model.add(Dense(1024, input_dim=3840, activation='relu'))
model.add(Dense(46, activation='softmax'))

In [25]:
# we have to compile our model before use -- using an Adam optimizer and 
# calculating loss using categorical crossentropy: 
model.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=['accuracy'])

### Fit the model.

In [26]:
# now we can fit our model! Let's just start with 10 epochs to keep things quick
model.fit(trainX, encoding_trainY, epochs=10, batch_size=100,verbose=True)

Epoch 1/10
14720/14720 [==============================] - 18s 1ms/step - loss: 1.3371 - acc: 0.6230
Epoch 2/10
14720/14720 [==============================] - 18s 1ms/step - loss: 0.5078 - acc: 0.8337
Epoch 3/10
14720/14720 [==============================] - 13s 912us/step - loss: 0.3613 - acc: 0.8586
Epoch 4/10
14720/14720 [==============================] - 14s 921us/step - loss: 0.3112 - acc: 0.8659
Epoch 5/10
14720/14720 [==============================] - 16s 1ms/step - loss: 0.3068 - acc: 0.8607
Epoch 6/10
14720/14720 [==============================] - 13s 907us/step - loss: 0.2747 - acc: 0.8647
Epoch 7/10
14720/14720 [==============================] - 13s 883us/step - loss: 0.2635 - acc: 0.8667
Epoch 8/10
14720/14720 [==============================] - 14s 922us/step - loss: 0.2617 - acc: 0.8663
Epoch 9/10
14720/14720 [==============================] - 16s 1ms/step - loss: 0.2517 - acc: 0.8686
Epoch 10/10
14720/14720 [==============================] - 16s 1ms/step - loss: 0.2517 - a

### How does our model perform?

In [27]:
# get accuracy on the test set
scores = model.evaluate(testX, encoding_testY)
print("\nAccuracy: %.2f%%" % (scores[1]*100))

3680/3680 [==============================] - 2s 440us/step

Accuracy: 87.04%


In [39]:
print(classification_report(testY,np.argmax(model.predict(testX),axis=1)))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00        78
          1       1.00      0.97      0.99        80
          2       0.97      1.00      0.99        75
          3       1.00      1.00      1.00        74
          4       1.00      1.00      1.00        70
          5       0.49      0.96      0.65        79
          6       1.00      1.00      1.00        86
          7       1.00      1.00      1.00        86
          8       1.00      1.00      1.00        84
          9       1.00      1.00      1.00        90
         10       1.00      0.97      0.99        75
         11       0.98      1.00      0.99        95
         12       0.50      0.99      0.67        75
         13       1.00      1.00      1.00        66
         14       1.00      1.00      1.00        71
         15       0.99      1.00      0.99        81
         16       1.00      0.99      0.99        76
         17       0.00      0.00      0.00   

In [40]:
# plot the confusion matrix
matrix(confusion_matrix(np.argmax(encoding_testY,axis=1),np.argmax(model.predict(testX),axis=1)))

(<toyplot.canvas.Canvas at 0x1821b204d0>,
 <toyplot.coordinates.Table at 0x1a2c00bdd0>)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 0 0.0 0.0 0.0 0.0 0.0 78.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1 0.0 78.0 2.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 2 0.0 0.0 75.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 3 0.0 0.0 0.0 74.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 4 0.0 0.0 0.0 0.0 70.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 5 0.0 0.0 0.0 0.0 0.0 76.0 0.0 0.0 0.0 0.0 0.0 0.0 2.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 6 0.0 0.0 0.0 0.0 0.0 0.0 86.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 7 0.0 0.0 0.0 0.0 0.0 0.0 0.0 86.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 8 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 84.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 9 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 90.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 10 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 73.0 2.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 11 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 95.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 12 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 74.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 13 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 66.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 14 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 71.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 15 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 81.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 16 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 75.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 17 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 71.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 18 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 86.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 19 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 80.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 20 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 84.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.

## Would adding some hidden layers change anything?

In [43]:
# Create a model -- lots of options in what we do here, except that the input dimensions
# and output dimensions are both fixed -- inputs at 1x3840, outputs at 1x46.
# This model has two hidden layers of 1024 nodes and two hidden layers of 512 nodes.
model = Sequential()
model.add(Dense(1024, input_dim=3840, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(46, activation='softmax'))

# we have to compile our model before use -- using an Adam optimizer and 
# calculating loss using categorical crossentropy: 
model.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=['accuracy'])

In [44]:
# now we can fit our model! Let's just start with 10 epochs to keep things quick
model.fit(trainX, encoding_trainY, epochs=10, batch_size=100,verbose=True)

Epoch 1/10
14720/14720 [==============================] - 28s 2ms/step - loss: 1.1828 - acc: 0.5910
Epoch 2/10
14720/14720 [==============================] - 27s 2ms/step - loss: 0.3977 - acc: 0.8226
Epoch 3/10
14720/14720 [==============================] - 21s 1ms/step - loss: 0.3498 - acc: 0.8351
Epoch 4/10
14720/14720 [==============================] - 24s 2ms/step - loss: 0.3329 - acc: 0.8380
Epoch 5/10
14720/14720 [==============================] - 26s 2ms/step - loss: 0.3070 - acc: 0.8437
Epoch 6/10
14720/14720 [==============================] - 24s 2ms/step - loss: 0.2822 - acc: 0.8564
Epoch 7/10
14720/14720 [==============================] - 24s 2ms/step - loss: 0.2926 - acc: 0.8523
Epoch 8/10
14720/14720 [==============================] - 25s 2ms/step - loss: 0.2773 - acc: 0.8577
Epoch 9/10
14720/14720 [==============================] - 25s 2ms/step - loss: 0.2908 - acc: 0.8525
Epoch 10/10
14720/14720 [==============================] - 25s 2ms/step - loss: 0.3122 - acc: 0.8475

In [45]:
# get accuracy on the test set
scores = model.evaluate(testX, encoding_testY)
print("\nAccuracy: %.2f%%" % (scores[1]*100))
# plot the confusion matrix
matrix(confusion_matrix(np.argmax(encoding_testY,axis=1),np.argmax(model.predict(testX),axis=1)))

3680/3680 [==============================] - 2s 606us/step

Accuracy: 86.58%


(<toyplot.canvas.Canvas at 0x1a34059d50>,
 <toyplot.coordinates.Table at 0x1a2f971650>)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 0 60.0 0.0 0.0 0.0 0.0 18.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1 0.0 75.0 5.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 2 0.0 0.0 75.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 3 0.0 0.0 0.0 74.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 4 0.0 0.0 0.0 0.0 70.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 5 60.0 0.0 0.0 0.0 0.0 19.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 6 0.0 0.0 0.0 0.0 0.0 0.0 83.0 3.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 7 0.0 0.0 0.0 0.0 0.0 0.0 0.0 86.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 8 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 84.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 9 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 90.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 10 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 73.0 2.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 11 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 95.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 12 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 8.0 0.0 0.0 0.0 0.0 66.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 13 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 66.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 14 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 71.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 15 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 67.0 14.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 16 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 76.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 17 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 69.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 18 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 86.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 19 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 80.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 20 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 80.0 0.0 0.0 0.0 0.0 4.0 0.0 0.0 0.0

## didn't change much, although it seems to differ a bit in which it predicts wrong...

## Conclusion:

1) Possibly need more training data.  
2) Might still be misspecifying model.  
3) Some of these might not be solveable -- maybe constrain parameters?

One cool outcome of this paper might be our ability to see under what conditions we might expect introgression to be undetectable from the data.

## Comparison to random forests:

This kind of problem might also make us think to use a treelike model

In [121]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(random_state=0,n_estimators=500)  

In [114]:
classifier.fit(trainX, trainY)  
y_pred = classifier.predict(testX)  

In [120]:
matrix(confusion_matrix(testY,y_pred)) 
print(accuracy_score(testY, y_pred)) 
print(classification_report(testY,y_pred))   

0.8790760869565217
             precision    recall  f1-score   support

          0       0.48      0.53      0.50        78
          1       1.00      0.97      0.99        80
          2       0.97      1.00      0.99        75
          3       1.00      1.00      1.00        74
          4       1.00      1.00      1.00        70
          5       0.47      0.44      0.46        79
          6       1.00      0.99      0.99        86
          7       0.99      1.00      0.99        86
          8       1.00      1.00      1.00        84
          9       1.00      1.00      1.00        90
         10       1.00      0.99      0.99        75
         11       0.99      1.00      0.99        95
         12       0.53      0.55      0.54        75
         13       1.00      1.00      1.00        66
         14       1.00      1.00      1.00        71
         15       1.00      1.00      1.00        81
         16       1.00      1.00      1.00        76
         17       0.53    

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 0 41.0 0.0 0.0 0.0 0.0 37.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1 0.0 78.0 2.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 2 0.0 0.0 75.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 3 0.0 0.0 0.0 74.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 4 0.0 0.0 0.0 0.0 70.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 5 42.0 0.0 0.0 0.0 0.0 35.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 6 0.0 0.0 0.0 0.0 0.0 0.0 85.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 7 0.0 0.0 0.0 0.0 0.0 0.0 0.0 86.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 8 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 84.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 9 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 90.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 10 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 74.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 11 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 95.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 12 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 41.0 0.0 0.0 0.0 0.0 33.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 13 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 66.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 14 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 71.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 15 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 81.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 16 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 76.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 17 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 32.0 0.0 0.0 0.0 0.0 39.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 18 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 86.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 19 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 80.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 20 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 82.0 0.0 0.0 0.0 0.0 2.0 0.0 0.0 0.

So random forests did just as good of a job here, and the distribution of incorrect predictions seems more consistent (it'll be cool to look at which specific branches these are).

I still think neural networks might be a better option once branch lengths / other parameters start changing.